In [1]:
!pip install xgboost shap
print("Packages downloaded successfully")

Packages downloaded successfully


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
morgan_dataset = pd.read_csv("~/datasets/morgan_dataset.csv")
ap_dataset = pd.read_csv("~/datasets/ap_dataset.csv")
rdk5_dataset = pd.read_csv("~/datasets/rdk5_dataset.csv")

In [4]:
# Ensure datasets are aligned (optional if already aligned)
morgan_dataset, ap_dataset = morgan_dataset.align(ap_dataset, join='inner', axis=0)
morgan_dataset, rdk5_dataset = morgan_dataset.align(rdk5_dataset, join='inner', axis=0)

# Split morgan_dataset into training and testing sets (80% train, 20% test)
morgan_train, morgan_test = train_test_split(morgan_dataset, test_size=0.2, random_state=42)

# Now split ap_dataset and rdk5_dataset based on the indices of the split
ap_train = ap_dataset.loc[morgan_train.index]  # Use the same indices from morgan_train
ap_test = ap_dataset.loc[morgan_test.index]  # Use the same indices from morgan_test

rdk5_train = rdk5_dataset.loc[morgan_train.index]  # Use the same indices from morgan_train
rdk5_test = rdk5_dataset.loc[morgan_test.index]  # Use the same indices from morgan_test

# Split based on 'activity' as the target
X_morgan_train_activity = morgan_train.drop(columns=['activity', 'pIC50'])
y_morgan_train_activity = morgan_train['activity']

X_morgan_test_activity = morgan_test.drop(columns=['activity', 'pIC50'])
y_morgan_test_activity = morgan_test['activity']

X_ap_train_activity = ap_train.drop(columns=['activity', 'pIC50'])
y_ap_train_activity = ap_train['activity']

X_ap_test_activity = ap_test.drop(columns=['activity', 'pIC50'])
y_ap_test_activity = ap_test['activity']

X_rdk5_train_activity = rdk5_train.drop(columns=['activity', 'pIC50'])
y_rdk5_train_activity = rdk5_train['activity']

X_rdk5_test_activity = rdk5_test.drop(columns=['activity', 'pIC50'])
y_rdk5_test_activity = rdk5_test['activity']

In [5]:
import os

# Expand `~` to the absolute home directory path
os.makedirs(os.path.expanduser("~/datasets/xgb"), exist_ok=True)

In [6]:
import os
import joblib
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define datasets and results directory
output_dir = os.path.expanduser("~/datasets/xgb")
os.makedirs(output_dir, exist_ok=True)

# Define dataset mappings
datasets = {
    "morgan": (X_morgan_train_activity, X_morgan_test_activity, y_morgan_train_activity, y_morgan_test_activity),
    "ap": (X_ap_train_activity, X_ap_test_activity, y_ap_train_activity, y_ap_test_activity),
    "rdk5": (X_rdk5_train_activity, X_rdk5_test_activity, y_rdk5_train_activity, y_rdk5_test_activity)
}

# Number of trials
num_trials = 40
results = []

# Run XGBoost training for each fingerprint type
for name, (X_train, X_test, y_train, y_test) in datasets.items():
    for trial in range(1, num_trials + 1):
        print(f"Trial {trial}/{num_trials} - Training XGBoost for {name} fingerprints...")
        
        # Initialize and train model
        model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=trial)
        model.fit(X_train, y_train)
        
        # Evaluate model
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Trial {trial} - {name} Accuracy: {accuracy:.4f}")
        
        # Save model
        model_path = os.path.join(output_dir, f"xgboost_{name}_trial{trial}.pkl")
        joblib.dump(model, model_path)

        # Run SHAP analysis
        print(f"Trial {trial} - Running SHAP analysis for {name} fingerprints...")
        explainer = shap.Explainer(model, X_test)
        shap_values = explainer(X_test)

        # Save SHAP values
        shap_path = os.path.join(output_dir, f"shap_values_{name}_trial{trial}.pkl")
        joblib.dump(shap_values, shap_path)

        # Save SHAP summary plot as image
        shap_plot_path = os.path.join(output_dir, f"shap_summary_{name}_trial{trial}.png")
        plt.figure()
        shap.summary_plot(shap_values, X_test, show=False)
        plt.savefig(shap_plot_path, bbox_inches="tight")
        plt.close()

        # Store results
        results.append({
            "trial": trial,
            "dataset": name,
            "accuracy": accuracy,
            "model_path": model_path,
            "shap_path": shap_path,
            "shap_plot_path": shap_plot_path
        })

# Convert results to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(output_dir, "xgboost_results.csv"), index=False)

print("All trials completed. Results and SHAP plots saved to:", output_dir)

Trial 1/40 - Training XGBoost for morgan fingerprints...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:28:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Trial 1 - morgan Accuracy: 0.9293
Trial 1 - Running SHAP analysis for morgan fingerprints...
Trial 2/40 - Training XGBoost for morgan fingerprints...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:28:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Trial 2 - morgan Accuracy: 0.9293
Trial 2 - Running SHAP analysis for morgan fingerprints...
Trial 3/40 - Training XGBoost for morgan fingerprints...


...

Trial 38 - rdk5 Accuracy: 0.9293
Trial 38 - Running SHAP analysis for rdk5 fingerprints...
Trial 39/40 - Training XGBoost for rdk5 fingerprints...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:38:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Trial 39 - rdk5 Accuracy: 0.9293
Trial 39 - Running SHAP analysis for rdk5 fingerprints...
Trial 40/40 - Training XGBoost for rdk5 fingerprints...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:38:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Trial 40 - rdk5 Accuracy: 0.9293
Trial 40 - Running SHAP analysis for rdk5 fingerprints...
All trials completed. Results and SHAP plots saved to: /teamspace/studios/this_studio/datasets/xgb
